# Window Functions

___

### Imports

In [1]:
import org.apache.spark.sql.SparkSession
val spark = SparkSessionkSession.builder().appName("window_functions").getOrCreate()

2020-01-29 11:39:38 WARN  SparkContext:66 - Using an existing SparkContext; some configuration may not take effect.


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@7ad9eda7


## Aggregate Functions

For aggregate functions, we can use the existing aggregate functions as window functions, e.g. `sum`, `avg`, `min`, `max` and `count`.

In [2]:
case class Salary(depName: String, empNo: Long, salary: Long)

defined class Salary


In [3]:
val empsalary = Seq(
  Salary("sales", 1, 5000),
  Salary("personnel", 2, 3900),
  Salary("sales", 3, 4800),
  Salary("sales", 4, 4800),
  Salary("personnel", 5, 3500),
  Salary("develop", 7, 4200),
  Salary("develop", 8, 6000),
  Salary("develop", 9, 4500),
  Salary("develop", 10, 5200),
  Salary("develop", 11, 5200)).toDS

empsalary: org.apache.spark.sql.Dataset[Salary] = [depName: string, empNo: bigint ... 1 more field]


In [4]:
empsalary

res0: org.apache.spark.sql.Dataset[Salary] = [depName: string, empNo: bigint ... 1 more field]


In [10]:
empsalary.show()

+---------+-----+------+
|  depName|empNo|salary|
+---------+-----+------+
|    sales|    1|  5000|
|personnel|    2|  3900|
|    sales|    3|  4800|
|    sales|    4|  4800|
|personnel|    5|  3500|
|  develop|    7|  4200|
|  develop|    8|  6000|
|  develop|    9|  4500|
|  develop|   10|  5200|
|  develop|   11|  5200|
+---------+-----+------+



#### Import for Window Function

In [6]:
import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.expressions.Window


In [8]:
// Windows are partitions of deptName
val byDepName = Window.partitionBy('depName)

byDepName: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@46801785


In [9]:
empsalary.withColumn("avg", avg('salary) over byDepName).show()

+---------+-----+------+-----------------+
|  depName|empNo|salary|              avg|
+---------+-----+------+-----------------+
|  develop|    7|  4200|           5020.0|
|  develop|    8|  6000|           5020.0|
|  develop|    9|  4500|           5020.0|
|  develop|   10|  5200|           5020.0|
|  develop|   11|  5200|           5020.0|
|    sales|    1|  5000|4866.666666666667|
|    sales|    3|  4800|4866.666666666667|
|    sales|    4|  4800|4866.666666666667|
|personnel|    2|  3900|           3700.0|
|personnel|    5|  3500|           3700.0|
+---------+-----+------+-----------------+



We describe a window using the convenient factory methods in Window object that create a window specification that you can further refine with **partitioning**, **ordering**, and **frame boundaries**.

After you describe a window you can apply window aggregate functions like **ranking** functions (e.g. `RANK`), **analytic** functions (e.g. `LAG`), and the regular aggregate functions, e.g. `sum`, `avg`, `max`.

**Note**

Window functions are supported in structured queries using SQL and Column-based expressions.

Although similar to aggregate functions, a window function does not group rows into a single output row and retains their separate identities. A window function can access rows that are linked to the current row.

**Note**

The main difference between window aggregate functions and aggregate functions with grouping operators is that the former calculate values for every row in a window while the latter gives you at most the number of input rows, one value per group.

We can mark a function window by `OVER` clause after a function in SQL, e.g. `avg(revenue) OVER (…​)` or over method on a function in the Dataset API, e.g. `rank().over(…​)`.

## Window object

Window object provides functions to define windows (as WindowSpec instances).

There are two families of the functions available in `Window` object that create WindowSpec instance for one or many Column instances:

- partitionBy
- orderBy

### Partitioning Records — `partitionBy` Methods

<code>
    partitionBy(colName: String, colNames: String*): WindowSpec
    partitionBy(cols: Column*): WindowSpec
</code>

`partitionBy` creates an instance of `WindowSpec` with partition expression(s) defined for one or more columns.

In [14]:
val tokens = Seq((0,"hello"),(1,"henry"),(2,"and"),(3,"harry")).toDF("id","token")

tokens: org.apache.spark.sql.DataFrame = [id: int, token: string]


In [15]:
tokens.show()

+---+-----+
| id|token|
+---+-----+
|  0|hello|
|  1|henry|
|  2|  and|
|  3|harry|
+---+-----+



In [16]:
// partition records into two groups
// * tokens starting with "h"
// * others

val byHTokens = Window.partitionBy('token startsWith "h")

byHTokens: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@38f7f9bf


In [17]:
// count the sum of ids in each group
val result = tokens.select('*, sum('id) over byHTokens as "sum over h tokens").orderBy('id)

result: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: int, token: string ... 1 more field]


In [19]:
result.show

+---+-----+-----------------+
| id|token|sum over h tokens|
+---+-----+-----------------+
|  0|hello|                4|
|  1|henry|                4|
|  2|  and|                2|
|  3|harry|                4|
+---+-----+-----------------+



## Ordering in Windows — `orderBy` Methods

<code>
    orderBy(colName: String, colNames: String*): WindowSpec
    orderBy(cols: Column*): WindowSpec
</code>

`orderBy` allows us to control the order of records in a window.

In [24]:
empsalary.show()

+---------+-----+------+
|  depName|empNo|salary|
+---------+-----+------+
|    sales|    1|  5000|
|personnel|    2|  3900|
|    sales|    3|  4800|
|    sales|    4|  4800|
|personnel|    5|  3500|
|  develop|    7|  4200|
|  develop|    8|  6000|
|  develop|    9|  4500|
|  develop|   10|  5200|
|  develop|   11|  5200|
+---------+-----+------+



In [20]:
val byDepnameSalaryDesc = Window.partitionBy('depname).orderBy('salary desc)

byDepnameSalaryDesc: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@22a9c1f3


In [21]:
// a numerical rank within the current row's partition for each distinct ORDER BY value
val rankByDepname = rank().over(byDepnameSalaryDesc)

rankByDepname: org.apache.spark.sql.Column = RANK() OVER (PARTITION BY depname ORDER BY salary DESC NULLS LAST unspecifiedframe$())


In [23]:
empsalary.select('*, rankByDepname as 'rank).show()

+---------+-----+------+----+
|  depName|empNo|salary|rank|
+---------+-----+------+----+
|  develop|    8|  6000|   1|
|  develop|   10|  5200|   2|
|  develop|   11|  5200|   2|
|  develop|    9|  4500|   4|
|  develop|    7|  4200|   5|
|    sales|    1|  5000|   1|
|    sales|    3|  4800|   2|
|    sales|    4|  4800|   2|
|personnel|    2|  3900|   1|
|personnel|    5|  3500|   2|
+---------+-----+------+----+



### `rangeBetween` Method

`
rangeBetween(start: Long, end: Long): WindowSpec
`

`rangeBetween` creates a WindowSpec with the frame boundaries from `start` (inclusive) to `end` (inclusive).

**Note**

It is recommended to use Window.unboundedPreceding, Window.unboundedFollowing and Window.currentRow to describe the frame boundaries when a frame is unbounded preceding, unbounded following and at current row, respectively.

In [25]:
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.expressions.WindowSpec
val spec: WindowSpec = Window.rangeBetween(Window.unboundedPreceding, Window.currentRow)

import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.expressions.WindowSpec
spec: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@3abb9a1f


Internally, `rangeBetween` creates a `WindowSpec` with SpecifiedWindowFrame and RangeFrame type.

At its core, a window function calculates a return value for every input row of a table based on a group of rows, called the frame. Every input row can have a unique **frame** associated with it.

When we define a frame you have to specify three components of a frame specification - the **start and end boundaries**, and the **type**.

Types of boundaries (two positions and three offsets):

 - `UNBOUNDED PRECEDING` - the first row of the partition

 - `UNBOUNDED FOLLOWING` - the last row of the partition

 - `CURRENT ROW`

 - `<value> PRECEDING`

 - `<value> FOLLOWING`

Offsets specify the offset from the current input row.

Types of frames:

 - ROW - based on *physical offsets* from the position of the current input row

 - RANGE - based on *logical offsets* from the position of the current input row

In the current implementation of WindowSpec you can use two methods to define a frame:

 - rowsBetween

 - rangeBetween

## Window Operators in SQL Queries

The grammar of windows operators in SQL accepts the following:

1. `CLUSTER BY` or `PARTITION BY` or `DISTRIBUTE BY` for partitions,

2. `ORDER BY` or `SORT BY` for sorting order,

3. `RANGE`, `ROWS`, `RANGE BETWEEN`, and `ROWS BETWEEN` for window frame types,

4. `UNBOUNDED PRECEDING`, `UNBOUNDED FOLLOWING`, `CURRENT ROW` for frame bounds.

### Examples

#### Top N per Group

Top N per Group is useful when you need to compute the first and second best-sellers in category.

#### `Question`: What are the best-selling and the second best-selling products in every category?

In [27]:
val dataset = Seq(
  ("Thin",       "cell phone", 6000),
  ("Normal",     "tablet",     1500),
  ("Mini",       "tablet",     5500),
  ("Ultra thin", "cell phone", 5000),
  ("Very thin",  "cell phone", 6000),
  ("Big",        "tablet",     2500),
  ("Bendable",   "cell phone", 3000),
  ("Foldable",   "cell phone", 3000),
  ("Pro",        "tablet",     4500),
  ("Pro2",       "tablet",     6500))
  .toDF("product", "category", "revenue")

dataset: org.apache.spark.sql.DataFrame = [product: string, category: string ... 1 more field]


In [28]:
dataset.show()

+----------+----------+-------+
|   product|  category|revenue|
+----------+----------+-------+
|      Thin|cell phone|   6000|
|    Normal|    tablet|   1500|
|      Mini|    tablet|   5500|
|Ultra thin|cell phone|   5000|
| Very thin|cell phone|   6000|
|       Big|    tablet|   2500|
|  Bendable|cell phone|   3000|
|  Foldable|cell phone|   3000|
|       Pro|    tablet|   4500|
|      Pro2|    tablet|   6500|
+----------+----------+-------+



In [30]:
dataset.where('category === "tablet").show()

+-------+--------+-------+
|product|category|revenue|
+-------+--------+-------+
| Normal|  tablet|   1500|
|   Mini|  tablet|   5500|
|    Big|  tablet|   2500|
|    Pro|  tablet|   4500|
|   Pro2|  tablet|   6500|
+-------+--------+-------+



The question boils down to ranking products in a category based on their revenue, and to pick the best selling and the second best-selling products based the ranking.

In [31]:
val overCategory = Window.partitionBy('category).orderBy('revenue.desc)

overCategory: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@100bb85c


In [35]:
val ranked = dataset.withColumn("dense_rank", dense_rank.over(overCategory))

ranked: org.apache.spark.sql.DataFrame = [product: string, category: string ... 2 more fields]


In [36]:
ranked.show()

+----------+----------+-------+----------+
|   product|  category|revenue|dense_rank|
+----------+----------+-------+----------+
|      Pro2|    tablet|   6500|         1|
|      Mini|    tablet|   5500|         2|
|       Pro|    tablet|   4500|         3|
|       Big|    tablet|   2500|         4|
|    Normal|    tablet|   1500|         5|
|      Thin|cell phone|   6000|         1|
| Very thin|cell phone|   6000|         1|
|Ultra thin|cell phone|   5000|         2|
|  Bendable|cell phone|   3000|         3|
|  Foldable|cell phone|   3000|         3|
+----------+----------+-------+----------+



### Revenue Difference per Category

In [37]:
val reveDesc = Window.partitionBy('category).orderBy('revenue.desc)

reveDesc: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@2d5a69a5


In [38]:
val reveDiff = max('revenue).over(reveDesc) - 'revenue

reveDiff: org.apache.spark.sql.Column = (max(revenue) OVER (PARTITION BY category ORDER BY revenue DESC NULLS LAST unspecifiedframe$()) - revenue)


In [39]:
dataset.select('*, reveDiff as 'revenue_diff).show()

+----------+----------+-------+------------+
|   product|  category|revenue|revenue_diff|
+----------+----------+-------+------------+
|      Pro2|    tablet|   6500|           0|
|      Mini|    tablet|   5500|        1000|
|       Pro|    tablet|   4500|        2000|
|       Big|    tablet|   2500|        4000|
|    Normal|    tablet|   1500|        5000|
|      Thin|cell phone|   6000|           0|
| Very thin|cell phone|   6000|           0|
|Ultra thin|cell phone|   5000|        1000|
|  Bendable|cell phone|   3000|        3000|
|  Foldable|cell phone|   3000|        3000|
+----------+----------+-------+------------+



### Difference on Column

#### Compute a difference between values in rows in a column.

In [40]:
val pairs = for {
  x <- 1 to 5
  y <- 1 to 2
} yield (x, 10 * x * y)

pairs: scala.collection.immutable.IndexedSeq[(Int, Int)] = Vector((1,10), (1,20), (2,20), (2,40), (3,30), (3,60), (4,40), (4,80), (5,50), (5,100))


In [42]:
val ds = pairs.toDF("ns", "tens")

ds: org.apache.spark.sql.DataFrame = [ns: int, tens: int]


In [43]:
ds.show()

+---+----+
| ns|tens|
+---+----+
|  1|  10|
|  1|  20|
|  2|  20|
|  2|  40|
|  3|  30|
|  3|  60|
|  4|  40|
|  4|  80|
|  5|  50|
|  5| 100|
+---+----+



In [44]:
val overNs = Window.partitionBy('ns).orderBy('tens)

overNs: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@7e315b8


In [45]:
val diff = lead('tens, 1).over(overNs)

diff: org.apache.spark.sql.Column = lead(tens, 1, NULL) OVER (PARTITION BY ns ORDER BY tens ASC NULLS FIRST unspecifiedframe$())


In [46]:
ds.withColumn("diff", diff - 'tens).show()

+---+----+----+
| ns|tens|diff|
+---+----+----+
|  1|  10|  10|
|  1|  20|null|
|  3|  30|  30|
|  3|  60|null|
|  5|  50|  50|
|  5| 100|null|
|  4|  40|  40|
|  4|  80|null|
|  2|  20|  20|
|  2|  40|null|
+---+----+----+



Please note that `Why do Window functions fail with "Window function X does not take a frame specification"`?

The key here is to remember that DataFrames are RDDs under the covers and hence aggregation like grouping by a key in DataFrames is RDD’s `groupBy` (or worse, `reduceByKey` or `aggregateByKey` transformations).

## Running Total

The **running total** is the sum of all previous lines including the current one.

In [47]:
val sales = Seq(
  (0, 0, 0, 5),
  (1, 0, 1, 3),
  (2, 0, 2, 1),
  (3, 1, 0, 2),
  (4, 2, 0, 8),
  (5, 2, 2, 8))
  .toDF("id", "orderID", "prodID", "orderQty")


sales: org.apache.spark.sql.DataFrame = [id: int, orderID: int ... 2 more fields]


In [48]:
sales.show()

+---+-------+------+--------+
| id|orderID|prodID|orderQty|
+---+-------+------+--------+
|  0|      0|     0|       5|
|  1|      0|     1|       3|
|  2|      0|     2|       1|
|  3|      1|     0|       2|
|  4|      2|     0|       8|
|  5|      2|     2|       8|
+---+-------+------+--------+



In [55]:
val orderedByID = Window.orderBy('id)
val totalQty = sum('orderQty).over(orderedByID).as('running_total)
val salesTotalQty = sales.select('*, totalQty).orderBy('id)

orderedByID: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@785f135d
totalQty: org.apache.spark.sql.Column = sum(orderQty) OVER (ORDER BY id ASC NULLS FIRST unspecifiedframe$()) AS `running_total`
salesTotalQty: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: int, orderID: int ... 3 more fields]


In [56]:
salesTotalQty.show()

2020-01-29 13:00:45 WARN  WindowExec:66 - No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+---+-------+------+--------+-------------+
| id|orderID|prodID|orderQty|running_total|
+---+-------+------+--------+-------------+
|  0|      0|     0|       5|            5|
|  1|      0|     1|       3|            8|
|  2|      0|     2|       1|            9|
|  3|      1|     0|       2|           11|
|  4|      2|     0|       8|           19|
|  5|      2|     2|       8|           27|
+---+-------+------+--------+-------------+



In [57]:
val byOrderId = orderedByID.partitionBy('orderID)
val totalQtyPerOrder = sum('orderQty).over(byOrderId).as('running_total_per_order)
val salesTotalQtyPerOrder = sales.select('*, totalQtyPerOrder).orderBy('id)

byOrderId: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@6b11c331
totalQtyPerOrder: org.apache.spark.sql.Column = sum(orderQty) OVER (PARTITION BY orderID ORDER BY id ASC NULLS FIRST unspecifiedframe$()) AS `running_total_per_order`
salesTotalQtyPerOrder: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: int, orderID: int ... 3 more fields]


In [58]:
salesTotalQtyPerOrder.show()

+---+-------+------+--------+-----------------------+
| id|orderID|prodID|orderQty|running_total_per_order|
+---+-------+------+--------+-----------------------+
|  0|      0|     0|       5|                      5|
|  1|      0|     1|       3|                      8|
|  2|      0|     2|       1|                      9|
|  3|      1|     0|       2|                      2|
|  4|      2|     0|       8|                      8|
|  5|      2|     2|       8|                     16|
+---+-------+------+--------+-----------------------+



### "Explaining" Query Plans of Windows

In [59]:
val byDepnameSalaryDesc = Window.partitionBy('depname).orderBy('salary desc)

byDepnameSalaryDesc: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@3ea89de6


##### Calculate rank of row

In [60]:
val rankByDepname = rank().over(byDepnameSalaryDesc)

rankByDepname: org.apache.spark.sql.Column = RANK() OVER (PARTITION BY depname ORDER BY salary DESC NULLS LAST unspecifiedframe$())


In [61]:
empsalary.show()

+---------+-----+------+
|  depName|empNo|salary|
+---------+-----+------+
|    sales|    1|  5000|
|personnel|    2|  3900|
|    sales|    3|  4800|
|    sales|    4|  4800|
|personnel|    5|  3500|
|  develop|    7|  4200|
|  develop|    8|  6000|
|  develop|    9|  4500|
|  develop|   10|  5200|
|  develop|   11|  5200|
+---------+-----+------+



In [62]:
empsalary.select('*, rankByDepname as 'rank).explain(extended = true)

== Parsed Logical Plan ==
'Project [*, rank() windowspecdefinition('depname, 'salary DESC NULLS LAST, unspecifiedframe$()) AS rank#1626]
+- AnalysisBarrier
      +- LocalRelation [depName#3, empNo#4L, salary#5L]

== Analyzed Logical Plan ==
depName: string, empNo: bigint, salary: bigint, rank: int
Project [depName#3, empNo#4L, salary#5L, rank#1626]
+- Project [depName#3, empNo#4L, salary#5L, rank#1626, rank#1626]
   +- Window [rank(salary#5L) windowspecdefinition(depname#3, salary#5L DESC NULLS LAST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS rank#1626], [depname#3], [salary#5L DESC NULLS LAST]
      +- Project [depName#3, empNo#4L, salary#5L]
         +- LocalRelation [depName#3, empNo#4L, salary#5L]

== Optimized Logical Plan ==
Window [rank(salary#5L) windowspecdefinition(depname#3, salary#5L DESC NULLS LAST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS rank#1626], [depname#3], [salary#5L DESC NULLS LAST]
+- LocalRelation [de

## `lag` Window Function

<code>
    lag(e: Column, offset: Int): Column
    lag(columnName: String, offset: Int): Column
    lag(columnName: String, offset: Int, defaultValue: Any): Column
    lag(e: Column, offset: Int, defaultValue: Any): Column
</code>

`lag` returns the value in `e / columnName` column that is `offset` records before the current record.

`lag` returns `null` value if the number of records in a window partition is less than `offset` or `defaultValue`.

In [63]:
val buckets = spark.range(9).withColumn("bucket", 'id % 3)

buckets: org.apache.spark.sql.DataFrame = [id: bigint, bucket: bigint]


In [65]:
buckets.show

+---+------+
| id|bucket|
+---+------+
|  0|     0|
|  1|     1|
|  2|     2|
|  3|     0|
|  4|     1|
|  5|     2|
|  6|     0|
|  7|     1|
|  8|     2|
+---+------+



In [66]:
// Make duplicates
val dataset = buckets.union(buckets)

dataset: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: bigint, bucket: bigint]


In [67]:
dataset.show()

+---+------+
| id|bucket|
+---+------+
|  0|     0|
|  1|     1|
|  2|     2|
|  3|     0|
|  4|     1|
|  5|     2|
|  6|     0|
|  7|     1|
|  8|     2|
|  0|     0|
|  1|     1|
|  2|     2|
|  3|     0|
|  4|     1|
|  5|     2|
|  6|     0|
|  7|     1|
|  8|     2|
+---+------+



In [68]:
val windowSpec = Window.partitionBy('bucket).orderBy('id)

windowSpec: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@4d9507db


In [69]:
dataset.withColumn("lag", lag('id, 1) over windowSpec).show()

+---+------+----+
| id|bucket| lag|
+---+------+----+
|  0|     0|null|
|  0|     0|   0|
|  3|     0|   0|
|  3|     0|   3|
|  6|     0|   3|
|  6|     0|   6|
|  1|     1|null|
|  1|     1|   1|
|  4|     1|   1|
|  4|     1|   4|
|  7|     1|   4|
|  7|     1|   7|
|  2|     2|null|
|  2|     2|   2|
|  5|     2|   2|
|  5|     2|   5|
|  8|     2|   5|
|  8|     2|   8|
+---+------+----+



In [70]:
dataset.withColumn("lag", lag('id, 2, "<default_value>") over windowSpec).show()

+---+------+----+
| id|bucket| lag|
+---+------+----+
|  0|     0|null|
|  0|     0|null|
|  3|     0|   0|
|  3|     0|   0|
|  6|     0|   3|
|  6|     0|   3|
|  1|     1|null|
|  1|     1|null|
|  4|     1|   1|
|  4|     1|   1|
|  7|     1|   4|
|  7|     1|   4|
|  2|     2|null|
|  2|     2|null|
|  5|     2|   2|
|  5|     2|   2|
|  8|     2|   5|
|  8|     2|   5|
+---+------+----+



**Caution**

It looks like `lag` with a default value has a bug — the default value’s not used at all.

## `lead` Window Function

<code>
    lead(columnName: String, offset: Int): Column
    lead(e: Column, offset: Int): Column
    lead(columnName: String, offset: Int, defaultValue: Any): Column
    lead(e: Column, offset: Int, defaultValue: Any): Column
</code>

`lead` returns the value that is `offset` records after the current records, and `defaultValue` if there is less than `offset` records after the current record. `lag` returns `null` value if the number of records in a window partition is less than `offset` or `defaultValue`.

In [71]:
dataset.withColumn("lead", lead('id, 1) over windowSpec).show()

+---+------+----+
| id|bucket|lead|
+---+------+----+
|  0|     0|   0|
|  0|     0|   3|
|  3|     0|   3|
|  3|     0|   6|
|  6|     0|   6|
|  6|     0|null|
|  1|     1|   1|
|  1|     1|   4|
|  4|     1|   4|
|  4|     1|   7|
|  7|     1|   7|
|  7|     1|null|
|  2|     2|   2|
|  2|     2|   5|
|  5|     2|   5|
|  5|     2|   8|
|  8|     2|   8|
|  8|     2|null|
+---+------+----+



In [72]:
dataset.withColumn("lead", lead('id, 2, "<default_value>") over windowSpec).show()

+---+------+----+
| id|bucket|lead|
+---+------+----+
|  0|     0|   3|
|  0|     0|   3|
|  3|     0|   6|
|  3|     0|   6|
|  6|     0|null|
|  6|     0|null|
|  1|     1|   4|
|  1|     1|   4|
|  4|     1|   7|
|  4|     1|   7|
|  7|     1|null|
|  7|     1|null|
|  2|     2|   5|
|  2|     2|   5|
|  5|     2|   8|
|  5|     2|   8|
|  8|     2|null|
|  8|     2|null|
+---+------+----+



**Caution**

It looks like `lead` with a default value has a bug — the default value’s not used at all.

## Cumulative Distribution of Records Across Window Partitions — `cume_dist` Window Function

`
cume_dist(): Column
`

`cume_dist` computes the cumulative distribution of the records in window partitions. This is equivalent to SQL’s `CUME_DIST` function.

In [73]:
dataset.withColumn("cume_dist", cume_dist over windowSpec).show()

+---+------+------------------+
| id|bucket|         cume_dist|
+---+------+------------------+
|  0|     0|0.3333333333333333|
|  0|     0|0.3333333333333333|
|  3|     0|0.6666666666666666|
|  3|     0|0.6666666666666666|
|  6|     0|               1.0|
|  6|     0|               1.0|
|  1|     1|0.3333333333333333|
|  1|     1|0.3333333333333333|
|  4|     1|0.6666666666666666|
|  4|     1|0.6666666666666666|
|  7|     1|               1.0|
|  7|     1|               1.0|
|  2|     2|0.3333333333333333|
|  2|     2|0.3333333333333333|
|  5|     2|0.6666666666666666|
|  5|     2|0.6666666666666666|
|  8|     2|               1.0|
|  8|     2|               1.0|
+---+------+------------------+



## Sequential numbering per window partition — `row_number` Window Function

`
row_number(): Column
`

`row_number` returns a sequential number starting at `1` within a window partition.

In [74]:
dataset.withColumn("row_number", row_number() over windowSpec).show()

+---+------+----------+
| id|bucket|row_number|
+---+------+----------+
|  0|     0|         1|
|  0|     0|         2|
|  3|     0|         3|
|  3|     0|         4|
|  6|     0|         5|
|  6|     0|         6|
|  1|     1|         1|
|  1|     1|         2|
|  4|     1|         3|
|  4|     1|         4|
|  7|     1|         5|
|  7|     1|         6|
|  2|     2|         1|
|  2|     2|         2|
|  5|     2|         3|
|  5|     2|         4|
|  8|     2|         5|
|  8|     2|         6|
+---+------+----------+



## `ntile` Window Function

`
ntile(n: Int): Column
`

`ntile` computes the ntile group id (from `1` to `n` inclusive) in an ordered window partition.

In [75]:
val dataset = spark.range(7).select('*, 'id % 3 as "bucket")

dataset: org.apache.spark.sql.DataFrame = [id: bigint, bucket: bigint]


In [76]:
dataset.show()

+---+------+
| id|bucket|
+---+------+
|  0|     0|
|  1|     1|
|  2|     2|
|  3|     0|
|  4|     1|
|  5|     2|
|  6|     0|
+---+------+



In [77]:
val byBuckets = Window.partitionBy('bucket).orderBy('id)

byBuckets: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@acf500b


In [78]:
dataset.select('*, ntile(3) over byBuckets as "ntile").show()

+---+------+-----+
| id|bucket|ntile|
+---+------+-----+
|  0|     0|    1|
|  3|     0|    2|
|  6|     0|    3|
|  1|     1|    1|
|  4|     1|    2|
|  2|     2|    1|
|  5|     2|    2|
+---+------+-----+



## Ranking Records per Window Partition — `rank` Window Function

`
rank(): Column
dense_rank(): Column
percent_rank(): Column
`

`rank` functions assign the sequential rank of each distinct value per window partition. They are equivalent to `RANK`, `DENSE_RANK` and `PERCENT_RANK` functions in the good ol' SQL.

`
rank(): Column
dense_rank(): Column
percent_rank(): Column
`

In [79]:
val dataset = spark.range(9).withColumn("bucket", 'id % 3)

dataset: org.apache.spark.sql.DataFrame = [id: bigint, bucket: bigint]


In [80]:
dataset.show()

+---+------+
| id|bucket|
+---+------+
|  0|     0|
|  1|     1|
|  2|     2|
|  3|     0|
|  4|     1|
|  5|     2|
|  6|     0|
|  7|     1|
|  8|     2|
+---+------+



In [81]:
val byBucket = Window.partitionBy('bucket).orderBy('id)

byBucket: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@11675596


#### rank

In [82]:
dataset.withColumn("rank", rank over byBucket).show()

+---+------+----+
| id|bucket|rank|
+---+------+----+
|  0|     0|   1|
|  3|     0|   2|
|  6|     0|   3|
|  1|     1|   1|
|  4|     1|   2|
|  7|     1|   3|
|  2|     2|   1|
|  5|     2|   2|
|  8|     2|   3|
+---+------+----+



#### dense_rank

In [83]:
dataset.withColumn("dense_rank", dense_rank over byBucket).show()

+---+------+----------+
| id|bucket|dense_rank|
+---+------+----------+
|  0|     0|         1|
|  3|     0|         2|
|  6|     0|         3|
|  1|     1|         1|
|  4|     1|         2|
|  7|     1|         3|
|  2|     2|         1|
|  5|     2|         2|
|  8|     2|         3|
+---+------+----------+



#### percent_rank

In [84]:
dataset.withColumn("percent_rank", percent_rank over byBucket).show()

+---+------+------------+
| id|bucket|percent_rank|
+---+------+------------+
|  0|     0|         0.0|
|  3|     0|         0.5|
|  6|     0|         1.0|
|  1|     1|         0.0|
|  4|     1|         0.5|
|  7|     1|         1.0|
|  2|     2|         0.0|
|  5|     2|         0.5|
|  8|     2|         1.0|
+---+------+------------+



`rank` function assigns the same rank for duplicate rows with a gap in the sequence (similarly to Olympic medal places). `dense_rank` is like `rank` for duplicate rows but compacts the ranks and removes the gaps.

In [85]:
// rank function with duplicates
// Note the missing/sparse ranks, i.e. 2 and 4
dataset.union(dataset).withColumn("rank", rank over byBucket).show

+---+------+----+
| id|bucket|rank|
+---+------+----+
|  0|     0|   1|
|  0|     0|   1|
|  3|     0|   3|
|  3|     0|   3|
|  6|     0|   5|
|  6|     0|   5|
|  1|     1|   1|
|  1|     1|   1|
|  4|     1|   3|
|  4|     1|   3|
|  7|     1|   5|
|  7|     1|   5|
|  2|     2|   1|
|  2|     2|   1|
|  5|     2|   3|
|  5|     2|   3|
|  8|     2|   5|
|  8|     2|   5|
+---+------+----+



In [86]:
// dense_rank function with duplicates
// Note that the missing ranks are now filled in
dataset.union(dataset).withColumn("dense_rank", dense_rank over byBucket).show

+---+------+----------+
| id|bucket|dense_rank|
+---+------+----------+
|  0|     0|         1|
|  0|     0|         1|
|  3|     0|         2|
|  3|     0|         2|
|  6|     0|         3|
|  6|     0|         3|
|  1|     1|         1|
|  1|     1|         1|
|  4|     1|         2|
|  4|     1|         2|
|  7|     1|         3|
|  7|     1|         3|
|  2|     2|         1|
|  2|     2|         1|
|  5|     2|         2|
|  5|     2|         2|
|  8|     2|         3|
|  8|     2|         3|
+---+------+----------+



In [87]:
// percent_rank function with duplicates
dataset.union(dataset).withColumn("percent_rank", percent_rank over byBucket).show

+---+------+------------+
| id|bucket|percent_rank|
+---+------+------------+
|  0|     0|         0.0|
|  0|     0|         0.0|
|  3|     0|         0.4|
|  3|     0|         0.4|
|  6|     0|         0.8|
|  6|     0|         0.8|
|  1|     1|         0.0|
|  1|     1|         0.0|
|  4|     1|         0.4|
|  4|     1|         0.4|
|  7|     1|         0.8|
|  7|     1|         0.8|
|  2|     2|         0.0|
|  2|     2|         0.0|
|  5|     2|         0.4|
|  5|     2|         0.4|
|  8|     2|         0.8|
|  8|     2|         0.8|
+---+------+------------+



### Closing Spark Session

In [88]:
spark.stop()